# English - Alice in wonderland

## Data preparation

In [6]:
from importlib import reload

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/pavlem/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [36]:
import preprocessor
preprocessor = reload(preprocessor)


pp = preprocessor.Preprocessor("data/original/alice_in_wonderland.txt", "eng", False)
cleaned_corpus = pp.process()
#pp.split(cleaned_corpus)

[later editions continued as follows When the sands are all dry, he is gay as a lark, And will talk in contemptuous tones of the Shark, But, when the tide rises and sharks are around, His voice has a timid and tremulous sound.].
[later editions continued as follows The Panther took pie-crust, and gravy, and meat, While the Owl had the dish as its share of the treat.
Number of unique characters: 35
Number of characters: 367
dict_keys(['[', 'l', 'a', 't', 'e', 'r', ' ', 'd', 'i', 'o', 'n', 's', 'c', 'u', 'f', 'w', 'W', 'h', 'y', ',', 'g', 'k', 'A', 'm', 'p', 'S', 'B', 'H', 'v', '.', ']', 'T', 'P', '-', 'O'])


## Subword segmentation

In [51]:
import sentpiece
sentpiece = reload(sentpiece)

# Character level train vocabular size = 68
# Character level test vocabular size = 68


NUM_CHARS = 68
TRAIN_DATA_PATH = "data/processed/eng_test.txt"
MODEL_NAME = "eng_model_chr"
TYPE = "_test"


sentpiece.train_model(TRAIN_DATA_PATH, MODEL_NAME + TYPE, NUM_CHARS)

In [45]:
!mv $MODEL_NAME* spm_models/

In [46]:
SEG_DATA_FILE = "eng_chr_test.txt"


sentpiece.segmentation(TRAIN_DATA_PATH, MODEL_NAME, SEG_DATA_FILE)

## Train LM

In [47]:
TRAIN_SEG_DATA_PATH = "data/segmented/eng_chr_train.txt"
TEST_SEG_DATA_PATH = "data/segmented/eng_chr_test.txt"
NUM_LAYERS = 40
CLASS = 9999
BPTT = 3


!bash train_script.sh $TRAIN_SEG_DATA_PATH $TEST_SEG_DATA_PATH $NUM_LAYERS $CLASS $BPTT

rm: cannot remove 'rnnlm/models/model': No such file or directory
rm: cannot remove 'rnnlm/models/model.output.txt': No such file or directory
debug mode: 2
train file: data/segmented/eng_chr_train.txt
valid file: data/segmented/eng_chr_test.txt
class size: 9999
Hidden layer size: 40
BPTT: 3
Rand seed: 1
rnnlm file: model
Starting training using file data/segmented/eng_chr_train.txt
Vocab size: 66
Words in train file: 113143
Iter:   0	Alpha: 0.100000	   TRAIN entropy: 3.1878    Words/sec: 424.3   VALID entropy: 3.2710
Iter:   1	Alpha: 0.100000	   TRAIN entropy: 2.7086    Words/sec: 425.2   VALID entropy: 3.0899
Iter:   2	Alpha: 0.100000	   TRAIN entropy: 2.5951    Words/sec: 429.8   VALID entropy: 3.0287
Iter:   3	Alpha: 0.100000	   TRAIN entropy: 2.5427    Words/sec: 693.5   VALID entropy: 2.9219
Iter:   4	Alpha: 0.100000	   TRAIN entropy: 2.5117    Words/sec: 908.4   VALID entropy: 2.8855
Iter:   5	Alpha: 0.100000	   TRAIN entropy: 2.4897    Words/sec: 929.5   VALID entropy: 2.8901
I

In [49]:
MODEL_OUTPUT = MODEL_NAME + ".output.txt"

!mv model $MODEL_NAME
!mv model.output.txt $MODEL_OUTPUT

!mv $MODEL_NAME $MODEL_OUTPUT rnnlm_models/

## Data generation

In [52]:
GEN_DATA_PATH = "data/generated/eng_model_chr/"
MODEL_PATH = f"./rnnlm_models/{MODEL_NAME}"

!bash gen_script.sh $GEN_DATA_PATH $MODEL_PATH

In [56]:
# Decoding generated texts

for i in range(1, 8):
    size = 10**i
    sentpiece.desegmentation(f"data/generated/eng_model_chr/{size}.txt", MODEL_NAME + TYPE)

# Bengali

# Analysis